### Factory Production Optimization

- Group 4: Huixin Li, Prashanti Bijapur, Roshni Victor, Shrutanvi Datar, Vrinda Sehgal

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Assumptions

- No inventory at the beggining of January.
- The machinery is already installed and the factory is in running since 10 years.

### Aim

- Our aim is to help the factory owner to optimize the usage of machinery inturn storing accurate amount of inventory so as to meet the optimal level of demand and supply.


### Sets and Indices

$n \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: Set of months.

$p \in \text{Products}=\{"Glassware","Home Decor",\dots,"Furniture"\}$: Set of products.

$m \in \text{Machines}=\{\text{Grinder},\text{VertDrill},\text{horiDrill},\text{Borer},\text{Planer}\}$: Set of machines.

### Decision Variables

$\text{Produce}_{n,p} \in \mathbb{R}^+$: Number of units of product $p$ to manufacture at month $n$.

$\text{Inventory}_{n,p} \in [0, \text{max_inventory}] \subset \mathbb{R}^+$: Number of units of product $p$ to store at month $n$.

$\text{Sold}_{n,p} \in [0, \text{max_sales}_{n,p}] \subset \mathbb{R}^+$: Number of units of product $p$ to sell at month $n$.

$\text{Maintenance}_{n,m} \in \{0,1,\dots, \text{down_req}_m\} \subset \mathbb{N}$: Number of machines of type $m$ scheduled for maintenance at month $n$.


### Constraints

- **Initial Balance:** For each product $p$, the number of units produced should be equal to the number of units sold plus the number stored (in units of product).

\begin{equation}
\text{Produce}_{\text{Jan},p} = \text{Sold}_{\text{Jan},p} + \text{Inventory}_{\text{Jan},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **Balance:** For each product $p$, the number of units produced in month $n$ and previously stored should be equal to the number of units sold and stored in that month (in units of product).

\begin{equation}
\text{Inventory}_{n-1,p} + \text{Produce}_{n,p} = \text{Sold}_{n,p} + \text{Inventory}_{n,p} \quad \forall (n,p) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Products}
\tag{2}
\end{equation}

- **Inventory Target:** The number of units of product $p$ kept in inventory at the end of the planning horizon should hit the target (in units of product).

\begin{equation}
\text{Inventory}_{\text{Jun},p} = \text{store_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}

- **Maintenance**: The number of machines of type $m$ scheduled for maintenance should meet the requirement.

\begin{equation}
\sum_{n \in \text{Months}}\text{Maintenance}_{n,m} = \text{down_req}_m \quad \forall m \in \text{Machines}
\tag{4}
\end{equation}

- **Machine Capacity:** Total time used to manufacture any product at machine type $m$ cannot exceed its monthly capacity (in hours).

\begin{equation}
\sum_{p \in \text{Products}}\text{time_req}_{m,p}*\text{Produce}_{n,p} \leq \text{hours_per_month}*(\text{installed}_m - \text{Maintenance}_{n,m}) \quad \forall (n,m) \in \text{Months} \times \text{Machines}
\tag{5}
\end{equation}



### Objective Function

- **Profit:** Maximize the total profit (in USD).

\begin{equation}
\text{Maximize} \quad T = \sum_{n \in \text{Months}}\sum_{p \in \text{Products}}
(\text{Profit}_p*\text{Produce}_{n,p} - \text{Inventory_cost}*\text{Inventory}_{n,p})
\
\end{equation}

In [2]:
# Parameters

products = ["Glassware", "Home Decor", "Plumbing", "Automotive", "Kitchen Appliances", "Electronics", "Furniture"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Glassware":10, "Home Decor":6, "Plumbing":8, "Automotive":4, "Kitchen Appliances":11, "Electronics":9, "Furniture":3}

time_req = {
    "grinder": {    "Glassware": 0.5, "Home Decor": 0.7, "Kitchen Appliances": 0.3,
                    "Electronics": 0.2, "Furniture": 0.5 },
    "vertDrill": {  "Glassware": 0.1, "Home Decor": 0.2, "Automotive": 0.3,
                    "Electronics": 0.6 },
    "horiDrill": {  "Glassware": 0.2, "Plumbing": 0.8, "Furniture": 0.6 },
    "borer": {      "Glassware": 0.05,"Home Decor": 0.03,"Automotive": 0.07,
                    "Kitchen Appliances": 0.1, "Furniture": 0.08 },
    "planer": {     "Plumbing": 0.01,"Kitchen Appliances": 0.05,"Furniture": 0.05 }
}

# number of each machine available
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# number of machines that need to be under maintenance
down_req = {"grinder":2, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# market limitation of sells
max_sales = {
    ("Jan", "Glassware") : 500,
    ("Jan", "Home Decor") : 1000,
    ("Jan", "Plumbing") : 300,
    ("Jan", "Automotive") : 300,
    ("Jan", "Kitchen Appliances") : 800,
    ("Jan", "Electronics") : 200,
    ("Jan", "Furniture") : 100,
    ("Feb", "Glassware") : 600,
    ("Feb", "Home Decor") : 500,
    ("Feb", "Plumbing") : 200,
    ("Feb", "Automotive") : 0,
    ("Feb", "Kitchen Appliances") : 400,
    ("Feb", "Electronics") : 300,
    ("Feb", "Furniture") : 150,
    ("Mar", "Glassware") : 300,
    ("Mar", "Home Decor") : 600,
    ("Mar", "Plumbing") : 0,
    ("Mar", "Automotive") : 0,
    ("Mar", "Kitchen Appliances") : 500,
    ("Mar", "Electronics") : 400,
    ("Mar", "Furniture") : 100,
    ("Apr", "Glassware") : 200,
    ("Apr", "Home Decor") : 300,
    ("Apr", "Plumbing") : 400,
    ("Apr", "Automotive") : 500,
    ("Apr", "Kitchen Appliances") : 200,
    ("Apr", "Electronics") : 0,
    ("Apr", "Furniture") : 100,
    ("May", "Glassware") : 0,
    ("May", "Home Decor") : 100,
    ("May", "Plumbing") : 500,
    ("May", "Automotive") : 100,
    ("May", "Kitchen Appliances") : 1000,
    ("May", "Electronics") : 300,
    ("May", "Furniture") : 0,
    ("Jun", "Glassware") : 500,
    ("Jun", "Home Decor") : 500,
    ("Jun", "Plumbing") : 100,
    ("Jun", "Automotive") : 300,
    ("Jun", "Kitchen Appliances") : 1100,
    ("Jun", "Electronics") : 500,
    ("Jun", "Furniture") : 60,
}

Inventory_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

#### Adding decision variables

In [3]:
factory = gp.Model('Factory Planning II')

# quantity manufactured
Produce = factory.addVars(months, products, vtype=GRB.INTEGER, name="Produce")

# quantity inventory stored
Inventory = factory.addVars(months, products, ub=max_inventory, vtype=GRB.INTEGER, name="Inventory") 

# quantity sold
Sold = factory.addVars(months, products, ub=max_sales, vtype=GRB.INTEGER, name="Sold") 

# number of machines under maintenance
Maintenance = factory.addVars(months, machines, vtype=GRB.INTEGER, ub=down_req, name="Maintain") 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-06


### Adding Constraints

#### 1. Initial Balance
For each product  𝑝 , the number of units produced should be equal to the number of units sold plus the number stored (in units of product).

In [4]:
Initial_Balance = factory.addConstrs((Produce[months[0], product] == Sold[months[0], product] 
                  + Inventory[months[0], product] for product in products), name="Initial_Balance")

#### 2. Balance
For each product  𝑝 , the number of units produced in month  𝑛  and previously stored should be equal to the number of units sold and stored in that month (in units of product).

In [ ]:
Balance = factory.addConstrs((Inventory[months[months.index(month) -1], product] + 
                Produce[month, product] == Sold[month, product] + Inventory[month, product] 
                for product in products for month in months 
                if month != months[0]), name="Balance")

#### 3. Inventory Target
The number of units of product  𝑝  kept in inventory at the end of the planning horizon should hit the target (in units of product).

In [5]:
Target_inventory = factory.addConstrs((Inventory[months[-1], product] == store_target for product in products),\
                                      name="Target_inventory")

#### 4. Machine Capacity
Total time used to manufacture any product at machine type  𝑚  cannot exceed its monthly capacity (in hours).

In [6]:
MachineCap = factory.addConstrs((gp.quicksum(time_req[machine][product] * Produce[month, product]
                             for product in time_req[machine])
                    <= hours_per_month * (installed[machine] - Maintenance[month, machine])
                    for machine in machines for month in months),
                   name = "MachineCap")

#### 5. Maintenance

The maintenance constraints ensure that the specified number and types of machines are down due maintenance in some month. Which month a machine is down is now part of the optimization.

In [7]:
Maintain = factory.addConstrs((Maintenance.sum('*', machine) == down_req[machine] for machine in machines), \
                              name="Maintain")

### Objective Function

In [8]:
obj = gp.quicksum(profit[product] * Sold[month, product] -  Inventory_cost * Inventory[month, product]  
               for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

#### Optimization and Gurobi Optimal Solution

In [9]:
factory.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 156 columns and 348 nonzeros
Model fingerprint: 0xcbc842ba
Variable types: 0 continuous, 156 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -175.0000000
Presolve removed 22 rows and 27 columns
Presolve time: 0.01s
Presolved: 62 rows, 129 columns, 278 nonzeros
Variable types: 0 continuous, 129 integer (12 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective 1.164550e+05, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 116455.000    0   10  


### Analysis

The maximum profit that can be achieved by considering all the constraints is $\$108,855.00$.

#### Production Plan
This plan determines the amount of each product to make at each period of the planning horizon.

In [11]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0)

for month, product in Produce.keys():
    if (abs(Produce[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(Produce[month, product].x, 1)
make_plan

,Glassware,Home Decor,Plumbing,Automotive,Kitchen Appliances,Electronics,Furniture
Jan,500,1000,300,300,800,200,100
Feb,600,500,200,0,400,300,150
Mar,400,700,100,100,600,400,200
Apr,0,0,0,0,0,0,0
May,0,100,500,100,1000,300,0
Jun,550,550,150,350,1150,550,110


#### Sales Plan
This plan defines the amount of each product to sell at each period of the planning horizon. 

In [12]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0)

for month, product in Sold.keys():
    if (abs(Sold[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(Sold[month, product].x, 1)
sell_plan

,Glassware,Home Decor,Plumbing,Automotive,Kitchen Appliances,Electronics,Furniture
Jan,500,1000,300,300,800,200,100
Feb,600,500,200,0,400,300,150
Mar,300,600,0,0,500,400,100
Apr,100,100,100,100,100,0,100
May,0,100,500,100,1000,300,0
Jun,500,500,100,300,1100,500,60


### Inventory Plan
This plan reflects the amount of products to be stored in inventory at the end of each period of the planning horizon.

In [13]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0)

for month, product in Inventory.keys():
    if (abs(Inventory[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(Inventory[month, product].x, 1)
store_plan

,Glassware,Home Decor,Plumbing,Automotive,Kitchen Appliances,Electronics,Furniture
Jan,0,0,0,0,0,0,0
Feb,0,0,0,0,0,0,0
Mar,100,100,100,100,100,0,100
Apr,0,0,0,0,0,0,0
May,0,0,0,0,0,0,0
Jun,50,50,50,50,50,50,50


### Maintenance Plan
This plan shows the machines to be under maintenance for each period of the planning horizon.

In [14]:
rows = months.copy()
columns = machines.copy()
repair_plan = pd.DataFrame(columns=columns, index=rows, data=0)

for month, machine in Maintenance.keys():
    if (abs(Maintenance[month, machine].x) > 1e-6):
        repair_plan.loc[month, machine] = Maintenance[month, machine].x
repair_plan

,grinder,vertDrill,horiDrill,borer,planer
Jan,0,0,0,0,0
Feb,0,0,0,0,0
Mar,0,0,1,0,0
Apr,2,1,0,1,1
May,0,1,0,0,0
Jun,0,0,2,0,0



## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.
